In [ ]:
!pip install scikit-tda

In [1]:
import os
import numpy as np
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
from glob import glob
from ripser import Rips

## Compute PD of PC

**WARNING!** It takes several hours to compute PDs so we provide PDs directly in `PD_pc` folder.

In [ ]:
rips = Rips()

In [4]:
test_data = np.load('./shapenet_test_18nn_GM_adaptive_knn_sparse.npy', allow_pickle=True)
test_label = test_data.tolist()['label']
test_data = test_data.tolist()['data']

In [ ]:
%%time
if not os.path.exists('PD_shapenet_test'):
    os.mkdir('PD_shapenet_test')

for i in tqdm_notebook(range(len(test_data))):
    data = test_data[i]

    dgm = rips.fit_transform(data)
    #np.savetxt('./PD_shapenet_test/0_' + str(i) + '.txt', np.array(dgm[0]), fmt='%s')
    np.savetxt('./PD_shapenet_test/1_' + str(i) + '.txt', np.array(dgm[1]), fmt='%s')

In [ ]:
train_data = np.load('./shapenet_train_18nn_GM_adaptive_knn_sparse.npy', allow_pickle=True)
train_label = train_data.tolist()['label']
train_data = train_data.tolist()['data']

In [ ]:
files = glob('PD_shapenet_train/1_*')
files = sorted(files, key=lambda x: int(x.split('/')[-1].split('.')[0].split('_')[1]))

In [ ]:
#idx = []
for i in tqdm_notebook(idx):
#    file = 'PD_shapenet_train/1_' + str(i) + '.txt'
#     if file not in files:
#         idx.append(i)
#         print(file)
    
    data = train_data[i]
    
    dgm = rips.fit_transform(data)
    np.savetxt('./PD_shapenet_train/0_' + str(i) + '.txt', np.array(dgm[0]), fmt='%s')
    np.savetxt('./PD_shapenet_train/1_' + str(i) + '.txt', np.array(dgm[1]), fmt='%s')

In [ ]:
%%time
if not os.path.exists('PD_shapenet_train'):
    os.mkdir('PD_shapenet_train')

for i in tqdm_notebook(range(len(train_data))):
    data = train_data[i]
    dgm = rips.fit_transform(data)
    np.savetxt('./PD_shapenet_train/0_' + str(i) + '.txt', np.array(dgm[0]), fmt='%s')
    np.savetxt('./PD_shapenet_train/1_' + str(i) + '.txt', np.array(dgm[1]), fmt='%s')

In [9]:
import multiprocessing
multiprocessing.cpu_count()

In [12]:
def f(i):
    data = test_data[i]
    dgm = rips.fit_transform(data)
    #np.savetxt('./PD_shapenet_test/0_' + str(i) + '.txt', np.array(dgm[0]), fmt='%s')
    np.savetxt('./PD_shapenet_test/1_' + str(i) + '.txt', np.array(dgm[1]), fmt='%s')

In [ ]:
# Get all cores
cores = 8 #multiprocessing.cpu_count()
# start a pool
pool = multiprocessing.Pool(processes=cores)
tasks = list(range(len(test_data)))
# do parallel calculate
print(pool.map(f, tasks))

In [3]:
# normalization
pd_files = glob.glob('./PD_pc/*.txt')
pd_files = sorted(pd_files)

In [5]:
# H0 normalization
max_h0 = 0
for i in range(len(pd_files)//2):
    data = np.loadtxt(pd_files[i])[:-1]  # disgarding inf 
    if np.max(data) > max_h0:
        max_h0 = np.max(data)

# H1 normalization
max_h1 = 0
for i in range(len(pd_files)//2):
    data = np.loadtxt(pd_files[len(pd_files)//2 + i])
    try:
        data[:, 1] = data[:, 1] - data[:, 0]
    except:
        data = data.reshape((1, 2))
        data[:, 1] = data[:, 1] - data[:, 0]
    if np.max(data) > max_h1:
        max_h1 = np.max(data) 

In [9]:
max_h0, max_h1

(0.7010728120803833, 1.0598217248916626)

In [31]:
# H0 normalization
for i in range(len(pd_files)//2):
    data = np.loadtxt(pd_files[i])[:-1]  # disgarding inf     
    data = data / max_h0
    np.savetxt('./PD_pc/' + pd_files[i].split('/')[-1], data, fmt='%s')

# H1 normalization
for i in range(len(pd_files)//2):
    data = np.loadtxt(pd_files[len(pd_files)//2 + i]) 
    try:
        data[:, 1] = data[:, 1] - data[:, 0]
    except:
        data = data.reshape((1, 2))
        data[:, 1] = data[:, 1] - data[:, 0]
    data = data / max_h1
    np.savetxt('./PD_pc/' + pd_files[len(pd_files)//2 + i].split('/')[-1], data, fmt='%s')        